In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel("bases/relatorio_ra-todas_17-12-2025.xlsx", engine="calamine", header=5)
df_lista = pd.read_excel("listagem/listagem.xlsx", engine="calamine")

In [ ]:
df.columns  

In [ ]:
df['Retorno Pedido'] = df['Retorno Pedido'].str.split(',').str[0]

In [ ]:
df["Data de Geração"].value_counts()

In [ ]:
df = df.merge(df_lista[['Cod. Loja', 'Franqueado']], how='left', on='Cod. Loja')

In [ ]:
df.isnull().sum()

In [ ]:
filtro_cencelada = df['Cancelada'] > 0

TD_cancelado = df[filtro_cencelada]


TD_cancelado = TD_cancelado.groupby('Retorno Pedido').agg(
    total_registros=('Retorno Pedido', 'size'),
    total_cancelados=('Cancelada', 'sum'),
    total_lojas=('Loja', 'nunique'),
    total_franqueados=('Franqueado', 'nunique')

).reset_index()



In [ ]:
total_geral = TD_cancelado['total_registros'].sum()
TD_cancelado['%proporção'] = TD_cancelado['total_registros'] / total_geral

linha_total = {
    'Retorno Pedido': 'TOTAL',
    'total_registros': TD_cancelado['total_registros'].sum(),
    'total_cancelados': TD_cancelado['total_cancelados'].sum(),
    'total_lojas': TD_cancelado['total_lojas'].sum(),
    'total_franqueados': TD_cancelado['total_franqueados'].sum(),
    '%proporção': 1.0


}

df_total = pd.DataFrame([linha_total])
td_final = pd.concat([TD_cancelado, df_total], ignore_index=True)

def fmt_milhar(valor):
    return f"{valor:,.0f}".replace(',', ".")

def fmt_pct(valor):
    return f"{valor:.0%}"

colunas_numericas = ['total_registros', 'total_cancelados', 'total_lojas', 'total_franqueados']

for col in colunas_numericas:
    td_final[col] = td_final[col].apply(fmt_milhar)


td_final['%proporção'] = td_final['%proporção'].apply(fmt_pct)

td_final.head()